In [ ]:
!pip install soundfile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split
int2emotion={
  'NEU':'neutral',
  'HAP':'happy',
  'SAD':'sad',
  'ANG':'angry',
  'FEA':'fearful',
  'DIS':'disgust',
  }
# Observed_Emotion
AVAILABLE_EMOTIONS=['neutral', 'happy', 'angry']

In [ ]:
def extract_feature(file_name, **kwargs):
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [ ]:
%cd '/content/drive/My Drive/speech_emotion/data'
!ls

/content/drive/My Drive/speech_emotion/data
A  Actor_01  Audio  AudioWAV  B  C


In [ ]:
def load_data(test_size=0.2):
    X, y = [], []
    try :
      for file in glob.glob('/content/drive/My Drive/speech_emotion/data/AudioWAV/*.wav'):
          # get the base name of the audio file
          basename = os.path.basename(file)
          print(basename)
          # get the emotion label
          emotion = int2emotion[basename.split("_")[2]]
          # we allow only AVAILABLE_EMOTIONS we set
          if emotion not in AVAILABLE_EMOTIONS:
              continue
          # extract speech features
          features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          # add to data
          X.append(features)
          l={'neutral':0.0,'happy':1.0, 'angry':2.0}
          y.append(l[emotion])
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [ ]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)

In [ ]:
import numpy as np
X_train = np.asarray(X_train)
y_train= np.asarray(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((2721, 180), (2721,), (908, 180), (908,))

In [ ]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [ ]:
x_traincnn.shape,x_testcnn.shape
#print(y_train[0:10])

((2721, 180, 1), (908, 180, 1))

In [ ]:
print(y_train[0:10])

[1. 2. 1. 0. 1. 2. 2. 1. 1. 0.]


In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import optimizers

model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))        #1
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))

model.add(Conv1D(128, 5,padding='same',))                           #2
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(3))                                                 #3
model.add(Activation('softmax'))
opt = optimizers.SGD(lr=0.00005, momentum=0.9, decay=0.0)

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_14 (Conv1D)           (None, 180, 128)          768       
_________________________________________________________________
activation_21 (Activation)   (None, 180, 128)          0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 180, 128)          0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 22, 128)           0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 22, 128)           82048     
_________________________________________________________________
activation_22 (Activation)   (None, 22, 128)           0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 22, 128)          

In [ ]:
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=1, epochs=500, validation_data=(x_testcnn, y_test))

Epoch 1/500
2721/2721 [==============================] - 14s 5ms/step - loss: 0.8292 - accuracy: 0.6825 - val_loss: 1.0502 - val_accuracy: 0.6233
Epoch 2/500
2721/2721 [==============================] - 10s 4ms/step - loss: 0.7451 - accuracy: 0.6964 - val_loss: 0.8691 - val_accuracy: 0.5936
Epoch 3/500
2721/2721 [==============================] - 11s 4ms/step - loss: 0.7232 - accuracy: 0.6898 - val_loss: 0.8357 - val_accuracy: 0.6509
Epoch 4/500
2721/2721 [==============================] - 10s 4ms/step - loss: 0.6896 - accuracy: 0.7031 - val_loss: 0.8722 - val_accuracy: 0.6388
Epoch 5/500
2721/2721 [==============================] - 10s 4ms/step - loss: 0.6744 - accuracy: 0.6983 - val_loss: 0.8080 - val_accuracy: 0.6586
Epoch 6/500
2721/2721 [==============================] - 11s 4ms/step - loss: 0.6702 - accuracy: 0.7064 - val_loss: 0.7912 - val_accuracy: 0.6509
Epoch 7/500
2721/2721 [==============================] - 10s 4ms/step - loss: 0.6605 - accuracy: 0.7218 - val_loss: 0.7614 -

In [ ]:
em=['neutral','happy','angry']

In [ ]:
predictions = model.predict_classes(x_testcnn)
n=predictions[1]
print(em[n])
print(n)

neutral
0


In [ ]:
loss, acc = model.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

29/29 [==============================] - 0s 7ms/step - loss: 1.8115 - accuracy: 0.6399
Restored model, accuracy: 63.99%


In [ ]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
predictions = model.predict_classes(x_testcnn)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.69      0.82      0.75       268
         1.0       0.61      0.59      0.60       337
         2.0       0.74      0.64      0.68       303

    accuracy                           0.67       908
   macro avg       0.68      0.68      0.68       908
weighted avg       0.68      0.67      0.67       908

[[220  43   5]
 [ 75 199  63]
 [ 25  85 193]]


In [ ]:
for file in glob.glob('/content/drive/My Drive/speech_emotion/03-01-02-02-01-01-01.wav'):
  features = np.array(extract_feature(file, mfcc=True, chroma=True, mel=True).reshape(1, -1))
    # predict
  f=np.expand_dims(features,axis=2)
  result = model.predict_classes(f)[0]
    # show the result !
  print("result :",em[result])

result : neutral


In [ ]:
pwd

'/content/drive/My Drive/speech_emotion/data'

In [ ]:
cd 

In [ ]:
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
from keras.models import model_from_json
json_file= open('/content/drive/My Drive/speech_emotion/data/model1.json','r')
loaded_model_json= json_file.read()
json_file.close()
Loaded_Model = model_from_json(loaded_model_json)
#Load weights into new model
Loaded_Model.load_weights("/content/drive/My Drive/speech_emotion/data/model1.h5")
print("Loaded")

Loaded


In [ ]:
neu = 0
hap = 0
ang = 0 
for file in glob.glob('/content/drive/My Drive/speech_emotion/SAD/*.wav'):
  features = np.array(extract_feature(file, mfcc=True, chroma=True, mel=True).reshape(1, -1))
    # predict
  f=np.expand_dims(features,axis=2)
  result = Loaded_Model.predict_classes(f)[0]
    # show the result !

  if (result == 0):
    neu = neu+1
  elif (result == 1):
    hap = hap+1
  else: ang = ang +1

  print("result: ",em[result])

print ('number neutral: ',neu)
print ('number happy: ',hap)
print ('number angry: ',ang)


result:  neutral
result:  happy
result:  happy
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  neutral
result:  happy
result:  happy
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  angry
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  happy
result:  happy
result:  happy
result:  neutral
result:  happy
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  happy
result:  neutral
result:  happy
result:  happy
result:  neutral
result:  happy
result:  neutral
result:  happy
result:  angry
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  happy
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


result:  angry
result:  happy
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  neutral
result:  happy
result:  happy
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  happy
result:  happy
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  happy
result:  neutral
result:  neutral
result:  happy
result:  happy
result:  neutral
result:  angry
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  neutral
result:  happy
result:  neutral
result:  neutral
result:  happy
result:  happy
res

In [ ]:
from IPython.display import Audio
from scipy.io import wavfile

def Testaudio(filepath):
  data = wavfile.read(filepath)
  framerate = data[0]
  sounddata = data[1]
  return Audio(sounddata,rate=framerate)

def PridictAudio(filepath):
  x=[]
  feature=extract_feature(filename, mfcc=True, chroma=True, mel=True)
  x.append(feature)
  predictions = model.predict_classes(x)
  #print('y: ',predictions)
  emotions=['neutral', 'happy', 'angry']
  print('Speech Recognition is: ',predictions[0])

In [ ]:
audio1 = '/content/drive/My Drive/speech_emotion/test/03a01Fa.wav'
audio2 = '/content/drive/My Drive/speech_emotion/data/B/03-01-04-01-01-01-01.wav'
PridictAudio(audio1)
Testaudio(audio1)

In [ ]:
file = "/content/drive/My Drive/speech_emotion/test/03a01Fa.wav"
feature=extract_feature(file, mfcc=True, chroma=True, mel=True)

RuntimeError: ignored